
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

In [4]:
# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [01:00<00:00,  1.64it/s]


In [2]:
# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [5]:
# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


/tmp/ipykernel_639/2447420142.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 1.9544639587402344
Epoch 10, Loss: 1.26720130443573
Epoch 20, Loss: 0.9196556210517883
Epoch 30, Loss: 0.7542466521263123
Epoch 40, Loss: 0.674754798412323
Epoch 50, Loss: 0.6291802525520325
Epoch 60, Loss: 0.599407970905304
Epoch 70, Loss: 0.5778950452804565
Epoch 80, Loss: 0.5613651871681213
Epoch 90, Loss: 0.5481103658676147
Epoch 99, Loss: 0.5381905436515808
Training complete!


## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

4. What would happen if we used directed edges instead of undirected edges for the random walks?
5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

### Question 1

What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?

If we increased the number of walks per node, the model could become more robust and learn the structure of the graph better. More training samples will allow the model to explore more of the neighborhood around each node and improve its embeddings. However, if the number of walks is already sufficiently high, increasing the number of walks would not improve the learned embeddings, as the entire neighborhood has already been explored, and the risk of overfitting comes into play. Let's test the model's performance on 400 walks instead of 200.

In [3]:
node2vec_num_walks = Node2Vec(G, dimensions=64, walk_length=30, num_walks=400, workers=2) 
model_num_walks = node2vec_num_walks.fit(window=10, min_count=1)
embeddings_num_walks = model_num_walks.wv

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [01:04<00:00,  3.08it/s]


In [4]:
classifier_num_walks = Classifier(64, 7)
optimizer_num_walks = optim.Adam(classifier_num_walks.parameters(), lr=0.01)

In [5]:
for epoch in range(100):
    classifier_num_walks.train()
    optimizer_num_walks.zero_grad()
    
    # Get node embeddings as input
    output = classifier_num_walks(torch.tensor([embeddings_num_walks[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer_num_walks.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

/tmp/ipykernel_1373/3569125100.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  output = classifier_num_walks(torch.tensor([embeddings_num_walks[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 2.1075894832611084
Epoch 10, Loss: 1.316653847694397
Epoch 20, Loss: 0.9489686489105225
Epoch 30, Loss: 0.7773062586784363
Epoch 40, Loss: 0.6985472440719604
Epoch 50, Loss: 0.6526168584823608
Epoch 60, Loss: 0.6219195127487183
Epoch 70, Loss: 0.5992518067359924
Epoch 80, Loss: 0.5816271305084229
Epoch 90, Loss: 0.5673202872276306
Epoch 99, Loss: 0.5564438700675964
Training complete!


The loss not changing much indicates that increasing the number of walks from 200 to 400 doesn't change much in the training of the model. The neighborhood of each node is explored enough with 200 nodes, so increasing to 400 doesn't help the model improve. If anything, it overfits the model, as seen by the loss slightly increasing.

### Question 2

What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

If we reduce the walk length, it could influence the information in the embeddings in multiple ways, depending on how well the current length (30) captures the information in the graph. If a length of 30 is greater than the "ideal" length (perhaps found through tuning the walk_length hyperparameter), then reducing the walk length would help create better embeddings. Perhaps 30 is too long, and the embeddings learned using 30 as the walk length are over-smoothed, so reducing walk length would help. Too high of a walk length causes the embeddings to lose locality, as they learn about nodes that are too far away to give useful information on classification. Conversely, if 30 is less than the ideal walk length, then the embeddings already have capacity to learn more about the graph structure, and reducing it further would only lose more useful information. Let's test it with a walk length of 20 instead of 30.

In [6]:
node2vec_walk_len = Node2Vec(G, dimensions=64, walk_length=20, num_walks=200, workers=2) 
model_walk_len = node2vec_walk_len.fit(window=10, min_count=1)
embeddings_walk_len = model_walk_len.wv

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:23<00:00,  4.26it/s]


In [7]:
classifier_walk_len = Classifier(64, 7)
optimizer_walk_len = optim.Adam(classifier_walk_len.parameters(), lr=0.01)

In [8]:
for epoch in range(100):
    classifier_walk_len.train()
    optimizer_walk_len.zero_grad()
    
    # Get node embeddings as input
    output = classifier_walk_len(torch.tensor([embeddings_walk_len[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer_walk_len.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Epoch 0, Loss: 1.9077131748199463
Epoch 10, Loss: 1.2275166511535645
Epoch 20, Loss: 0.8860049247741699
Epoch 30, Loss: 0.741150438785553
Epoch 40, Loss: 0.6704614758491516
Epoch 50, Loss: 0.6283754110336304
Epoch 60, Loss: 0.5997745394706726
Epoch 70, Loss: 0.5786681175231934
Epoch 80, Loss: 0.5622506737709045
Epoch 90, Loss: 0.5489523410797119
Epoch 99, Loss: 0.538888692855835
Training complete!


Since the loss remained the same with a walk length of 20, this indicates that, like the number of walks, the difference between 20 and 30 walk length isn't enough to capture any new information about the data. This could also be a result of loops and clusters in the graph, where increasing the walk length only just goes around the same loop or cluster an extra time, without adding information to the node embeddings.

### Question 3

What would happen if we used directed edges instead of undirected edges for the random walks?

The walks and embeddings would be completely different. The paths found by the algorithm would change to follow the direction of the edges instead of going in any direction. There is an argument that this is a better representation of the dataset, as papers don't cite each other, so indicating the direction of citation is useful. However, since we are trying to find similarity between papers, a node embedding losing information from the papers that cite that paper could result in a worse model. Let's test the model performance on a directed graph.

In [3]:
G_dir = to_networkx(data, to_undirected=False)
node2vec_dir = Node2Vec(G_dir, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model_dir = node2vec_dir.fit(window=10, min_count=1)
embeddings_dir = model_dir.wv

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:32<00:00,  3.12it/s]


In [4]:
classifier_dir = Classifier(64, 7)
optimizer_dir = optim.Adam(classifier_dir.parameters(), lr=0.01)

In [5]:
for epoch in range(100):
    classifier_dir.train()
    optimizer_dir.zero_grad()
    
    # Get node embeddings as input
    output = classifier_dir(torch.tensor([embeddings_dir[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer_dir.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

/tmp/ipykernel_170/4156179803.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  output = classifier_dir(torch.tensor([embeddings_dir[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 1.891316533088684
Epoch 10, Loss: 1.2305419445037842
Epoch 20, Loss: 0.8907303214073181
Epoch 30, Loss: 0.741433322429657
Epoch 40, Loss: 0.6700621843338013
Epoch 50, Loss: 0.6287826299667358
Epoch 60, Loss: 0.6011267304420471
Epoch 70, Loss: 0.5809458494186401
Epoch 80, Loss: 0.5653074979782104
Epoch 90, Loss: 0.5526865720748901
Epoch 99, Loss: 0.5431882739067078
Training complete!


In terms of loss, this model performed basically the same as the baseline model with the undirected graph. This would indicate that directionality has no effect on the learning of embeddings in this case. Especially in the case of loops or clusters, if closely related nodes are already very connected through their citations, changing the directionality of the graph doesn't make it easier or more difficult for the random walk to represent graph structure.

### Question 4

What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?

**Answer:** The feature vector of size 1433 in the Cora dataset is a bag-of-words vector, with each entry indicating if the word that entry represents is present in the paper. Increasing the number of words deemed significant from 1433 to 2000 could allow the model to find more nuanced relationships between the papers, and better able to determine graph structure for the embeddings, but it could also lead to overfitting. If the additional words are chosen properly to be meaningful words indicating similarity in scientific writing, the model's performance would improve. However, adding any words to the bag-of-words could include trivial words that appear in everyday language and writing. This causes the model to find relationships between papers based on authors' style of writing and the similar words they use, not indicating similarity in content of the research. Similarly, if any of the additional 567 words include stopwords - such as and, the, but, for, etc - it would only exaggerate the issue.

### Question 5

What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?

**Answer:** Adding more classes would make the Random Walk GNN classification problem more difficult, and performance would likely change. The embeddings can only hold so much information, and if there are too many classes for the size of the embedding, then the difference between classes can become indistinguishable, and the classifier will struggle to make accurate predictions. Increasing the embedding dimension could help mitigate this issue to help performace, but in general, more classes would cause the classifier to become less accurate.

### Question 6

What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?

In [6]:
node2vec128 = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=2) 
model128 = node2vec128.fit(window=10, min_count=1)
embeddings128 = model128.wv

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:43<00:00,  2.30it/s]


In [7]:
classifier128 = Classifier(128, 7)
optimizer128 = optim.Adam(classifier128.parameters(), lr=0.01)

In [8]:
for epoch in range(100):
    classifier128.train()
    optimizer128.zero_grad()
    
    # Get node embeddings as input
    output = classifier128(torch.tensor([embeddings128[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer128.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Epoch 0, Loss: 1.934342622756958
Epoch 10, Loss: 1.064994215965271
Epoch 20, Loss: 0.7446576952934265
Epoch 30, Loss: 0.6353906393051147
Epoch 40, Loss: 0.5796465277671814
Epoch 50, Loss: 0.542105495929718
Epoch 60, Loss: 0.5147106647491455
Epoch 70, Loss: 0.4936724007129669
Epoch 80, Loss: 0.47655636072158813
Epoch 90, Loss: 0.46208810806274414
Epoch 99, Loss: 0.4507688283920288
Training complete!


While the training time of the model slightly increased, the model's performace improved. The loss after 100 epochs of 0.45 is a noticable improvement over the loss of 0.53 that the baseline model produced. The larger embedding dimension allows the model to capture more features and decrease the loss.

### Extra Credit 1

What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

Increasing the window size increases the number of neighboring nodes considered for context. Higher window size can improve the model and reduce loss, but could introduce too much noise and lead to over-smoothing. Let's test what happens when increasing window size to 15.

In [3]:
node2vec_window = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model_window = node2vec_window.fit(window=15, min_count=1)
embeddings_window = model_window.wv

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [4]:
classifier_window = Classifier(64, 7)
optimizer_window = optim.Adam(classifier_window.parameters(), lr=0.01)

In [5]:
for epoch in range(100):
    classifier_window.train()
    optimizer_window.zero_grad()
    
    # Get node embeddings as input
    output = classifier_window(torch.tensor([embeddings_window[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer_window.step()

    if (epoch % 10 == 0) or (epoch == 99):
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

/tmp/ipykernel_160/3794119665.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  output = classifier_window(torch.tensor([embeddings_window[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 1.971235752105713
Epoch 10, Loss: 1.2834528684616089
Epoch 20, Loss: 0.9324180483818054
Epoch 30, Loss: 0.7695595622062683
Epoch 40, Loss: 0.6894205808639526
Epoch 50, Loss: 0.6443541646003723
Epoch 60, Loss: 0.6143700480461121
Epoch 70, Loss: 0.5927897095680237
Epoch 80, Loss: 0.5762441158294678
Epoch 90, Loss: 0.562949001789093
Epoch 99, Loss: 0.5529383420944214
Training complete!


Increasing the window size also doesn't have much of an effect of loss, likely due to the structure of the graph. If what I said earlier about the loops and clusters is true, increasing the window size would also give redundant information and not help to improve the model. If anything, it reduced the embedding quality by allowing the random walk to search too far, and either adding redundant information from nearby nodes or unnecessary information from far away nodes. Either way, increasing the window size did not improve the embeddings.